# Introduction

In class we discussed a multitude of reinforcement learning (RL) algorithms, methodologies, concepts, etc. My goal with this notebook is to review that information and further organize it. At the end of this I hope to have a good understanding of the landscape of RL, and insight into when to apply one method versus another. I will begin by discussing key concepts that apply across all of RL. Next, I will give a high-level overview of methods in RL. It is not my intention to be entirely exhaustive nor completely mathematically rigorous, but to give a practical guide for myself.

To conduct this exploration I will be using class notes, [OpenAI Spinning Up](https://spinningup.openai.com/en/latest/index.html), and other internet resources cited as necessary.

# RL Concepts

I will not re-formulate the entire problem of reinforcement learning, but I will review key ideas and notation.

## Notation

In any RL setting I would say there are seven foundational concepts: environment, agent, state space, action space, observations, policies, and rewards. The environment and agent are both quite straightforward, with the environment defining the world and the agent acting in that world. In a general RL loop an agent observes the state of the environment, performs some kind of action according to a policy that likely affects the environment, and then recieves a reward for this action. This process repeats as the agent then observes the altered environment and performs another action.

It is important to note that the state and observation are not necessarily the same. The state is an omniscient description of the environment whereas an observation may only be a partial description. As well, actions may be discrete or continuous and are defined via a set of legal actions an agent may take. A policy maps an observation to an action and may be deterministic or stochastic. The latter case can be due to noise or uncertainity in the environment or agent, or it could be deliberate to encourage exploration.

We define the following notation used throughout:

$$
\begin{align}
    & \mathcal{S}\rightarrow\text{State Space}\\
    & \mathcal{X}\rightarrow\text{Observations}\\
    & \mathcal{A}\rightarrow\text{Action Space}\\
    & \pi\rightarrow\text{Policy},\:\pi:\mathcal{X}\to\mathcal{A}\\
    & r\rightarrow\text{Reward function},\: r:\mathcal{S}x\mathcal{A}x\mathcal{S}\to\mathbb{R}
\end{align}
$$

I will note that I somewhat break with the standard notation in writing the policy as mapping from observations and not states. I do this to make it clear that the two are different, and it seems to me that an agent could be in the same state but have different observations. Thus it would be deceptive to say the policy maps from states, as in reality the same state could map to multiple actions. However, I do define the reward function as mapping from states because a reward can be independent from the agents knowledge of the world. I also note that the reward function can be (but isn't necessarily) dependent on the current state, current action, and future state.

The goal of RL given this setup is to find the policy that maximizes the expected return defined as the (possibly discounted) expected cumulative future reward. Note that this future reward could be infinite or finite horizon. In the following we will not specify which we consider unless it is necessary, and we define the future return as $R$.

## Value Functions

Value functions are a useful concept that give a metric of how good (i.e. how valuable) a state or state-action pair are. The value function defines the expected future return given the agent starts in state $s$ and acts according to policy $\pi$:

$$ V^\pi(s)=E_{\pi}[R|s_0=s] $$

The action-value function -- also known as the Q-function -- defines the expected furture return given the agent starts in state $s$, takes action $a$, and then acts according to policy $\pi$.

$$ Q^\pi(s,a)=E_{\pi}[R|s_0=s,a_0=a] $$

I note that I start time from zero here even though that may not be the case, but it can always be shifted such that it does.

The optimal versions of these functions are simply defined as the max over all possible policies. This is important because if we know the optimal action-value function then we would know exactly how to act in any state: simply take the action the maximizes the optimal Q function.

## Bellman Equations


The Bellman equations are ways of rewriting the value and action-value functions seen above. It often comes up in RL and can be directly used to define learning algorithms. The Bellman equation for the value function can simply be thought of as expanding the $R$ term into the current reward and the value function for the next state. We introduce a few more pieces of notation here; namely, the environment $\varepsilon$ and the discount factor $\gamma$.

$$ V^\pi(s) = E_{a\sim\pi,s'\sim\varepsilon}[r(s,a)+\gamma V^\pi(s')] $$

Similarily, for the action value function we expand the $R$ term into the current reward and the future value function for the next state. Here, we note that the value function can be defined in terms of the action value function: $V^\pi(s)=E_{a\sim\pi}[Q^\pi(s,a)]$. This follows because in the case of the value function we are picking the first action from the policy. The Bellman equation here is then given as follows:

$$ Q^\pi(s,a)=E_{s'\sim\varepsilon}[r(s,a)+\gamma E_{a'\sim\pi}[Q^\pi(s',a')]]$$

## Neural Networks?

All of what we have worked through is useful and interesting, but it is independent of deep learning. The question is where do we add a neural network to leverage their capabilities? There are multiple answers to this question, and potentially new avenues to explore, but one such answer is the Q function.

$$ Q(s,a;\theta)\approx Q^*(s,a) $$

Which is a network with parameters $\theta$ that approximates the optimal action-value function. Again there is much more that can be done here, but we will look into that more in the sections below.

## Experience Replay

An important concept in the setting of deep RL is that of experience replay. In order to train an agent we need it it to interact with the environment and gain experience. Without going into too much detail we know that we want our training data to be iid, and if we use our sequential environment interactions to learn we likely don't maintain that. Instead, we can store past experiences in a set and sample randomly from that set anytime we want to learn. This helps us ensure the iid assumption, and it can have other benefits as well, like replaying rare events.

# RL Algorithms

In this section I will explore a broader overview of RL methods. Again, I will not cover everything there is to know, nor is it entirely possible to perfectly catagorize everything.

## High-Level

There are two sets of terms which are central to RL methods that I wish to define and discuss: model-based versus model-free and on-policy versus off-policy.

Model-based and model-free refer to whether or not the agent has access to a model of the environment. For example, we can imagine an agent learning to play chess that has a model of how chess works. This is model-based as the agent has an idea of how actions will affect the environment, in this example through the rules of chess. This model allows an agent to plan multiple steps ahead as we have seen in the Go papers discussed in class. I will also note that the model may be something that exists by default in the agent, or it can be learned. On the other hand model-free means the agent does not know how environment will respond and cannot plan ahead. They both have their benefits, but certainly present a distinct difference in methods.

On-policy and off-policy refer to how an agent uses its experience during optimization. In the "on" case the agent optimizes the policy it is using to make decisions. This means an agent makes a decision according to its policy, realizes that decision, and updates its policy accordingly. In the "off" case the agent can use any data from its experiences even if that data didn't come from executing its current policy. Learning off-policy allows the agent to explore, to make greedy decisions, or otherwise pursue other options not allowed by its current policy. Again, there are benefits to both methods and sometimes a method can exhibit both behaviours.

Two of the biggest categories in model-free RL are Policy Optimization and Q-Learning. As mentioned above these can sometimes cross-over, but it provides a good starting point.

In policy optimization we model the policy directly using a neural network. This gives us $\pi_\theta(a|s)$ where $\theta$ are the parameters of the network, so we give it a state and it gives us an action. In order to optimize the network we often need to use another network that models the value function ($V_\phi(s)$). This is where the name for Actor-Critic type methods comes from, as we have a network that acts and one that evaluates.

In Q-Learning we model the Q-function using a neural network, as discussed earlier. In this case we can use the Bellman equation to iteratively update parameters. When we want to extract an action to implement given a current state we simply select the action that maximizes the optimal Q-function which is modeled by our network.

Clearly there are some considerations that need to be made about continuous vs discrete action spaces for this methods. For example, if we had a continuous action space we wouldn't be able to simply select the maximizing action from our Q-network.

There are many more classes of methods, but we will end our discussion here.